<a href="https://colab.research.google.com/github/charlotteesharpp/IMLO-project/blob/main/FlowerClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting to google drive in order to save and load the model features

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


All imports needed

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import time

Transformations for loading the data into sets:
* Training: augmentation to improve the accuracy
* Validation and Testing: no augemtation

In [3]:
# Define the transform to preprocess the images for the training data
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(15),  # Randomly rotate images in the range
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  #chnage the brightness
    transforms.RandomCrop(224, padding=4, padding_mode='reflect'),  # Randomly crop images padded by 4 pixels on each side
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalise the images
])

#for the test and validation sets
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Load in the data, apply the transformations and place in dataloaders

In [4]:
# Load the dataset (you will need to specify the path to your dataset)
train_dataset = datasets.Flowers102(
    root="data",
    split="train",
    transform=train_transform,
    download=True,
)
test_dataset = datasets.Flowers102(
    root="data",
    split="test",
    transform=test_transform,
    download=True,
)

val_dataset = datasets.Flowers102(
    root="data",
    split="val",
    transform=test_transform,
    download=True,
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

100%|██████████| 344862509/344862509 [00:11<00:00, 30139485.59it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 458423.82it/s]


100%|██████████| 14989/14989 [00:00<00:00, 5362369.73it/s]


Define a CNN architecture

In [5]:
class FlowerClassifier(nn.Module):
    def __init__(self):
        super(FlowerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 28 * 28, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 102)

    def forward(self, x):
        x = self.bn1(torch.relu(self.conv1(x)))
        x = self.pool(x)
        x = self.bn2(torch.relu(self.conv2(x)))
        x = self.pool(x)
        x = self.bn3(torch.relu(self.conv3(x)))
        x = self.pool(x)
        x = x.view(-1, 128 * 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Initialise the model, loss function, and optimiser

In [6]:
model = FlowerClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

Define training, validating and testing algoirthms

In [7]:
def train_model(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}]: Loss: {loss.item():.4f}")

def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
    return val_loss

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for test_images, test_labels in test_loader:
            test_outputs = model(test_images)
            _, predicted = torch.max(test_outputs, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()
    accuracy = correct / total
    return accuracy

begin training the model

In [8]:
num_epochs = 15
start_time = time.time()

for epoch in range(num_epochs):
        train_model(model, train_loader, optimizer, criterion, epoch)
        if epoch % 5 == 0:
            val_loss = validate_model(model, val_loader, criterion)
            print(f"Validation Loss at Epoch {epoch}: {val_loss:.4f}")

test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy (top-1): {100 * test_accuracy:.2f}%")
print(f"Finished Training. Total elapsed time: {round((time.time() - start_time) / 60, 1)} min")

Epoch [0/15]: Loss: 7.0279
Validation Loss at Epoch 0: 208.8797
Epoch [1/15]: Loss: 5.9418
Epoch [2/15]: Loss: 4.6816
Epoch [3/15]: Loss: 3.5377
Epoch [4/15]: Loss: 3.2235
Epoch [5/15]: Loss: 2.8179
Validation Loss at Epoch 5: 103.3799
Epoch [6/15]: Loss: 2.5254
Epoch [7/15]: Loss: 2.0479
Epoch [8/15]: Loss: 2.3801
Epoch [9/15]: Loss: 1.8011
Epoch [10/15]: Loss: 1.9484
Validation Loss at Epoch 10: 89.6794
Epoch [11/15]: Loss: 1.6636
Epoch [12/15]: Loss: 1.1849
Epoch [13/15]: Loss: 1.4694
Epoch [14/15]: Loss: 1.3909
Test Accuracy (top-1): 31.18%
Finished Training. Total elapsed time: 85.8 min


Save the model

In [9]:
torch.save(model.state_dict(), "/content/drive/MyDrive/models/flowersv1")
print("Saved PyTorch Model State to path")

Saved PyTorch Model State to path


training 1:
* Test Accuracy (top-1): 31.18%
* Total elapsed time: 85.8 min

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = FlowerClassifier().to(device)
net.load_state_dict(torch.load("/content/drive/MyDrive/models/flowersv1"))

<All keys matched successfully>

In [11]:
num_epochs = 15
start_time = time.time()

for epoch in range(num_epochs):
        train_model(model, train_loader, optimizer, criterion, epoch)
        if epoch % 5 == 0:
            val_loss = validate_model(model, val_loader, criterion)
            print(f"Validation Loss at Epoch {epoch}: {val_loss:.4f}")

test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy (top-1): {100 * test_accuracy:.2f}%")
print(f"Finished Training. Total elapsed time: {round((time.time() - start_time) / 60, 1)} min")

Epoch [0/15]: Loss: 1.1101
Validation Loss at Epoch 0: 84.2772
Epoch [1/15]: Loss: 1.0137
Epoch [2/15]: Loss: 1.1198
Epoch [3/15]: Loss: 1.1862
Epoch [4/15]: Loss: 0.7653
Epoch [5/15]: Loss: 0.9504
Validation Loss at Epoch 5: 81.4847
Epoch [6/15]: Loss: 0.8602
Epoch [7/15]: Loss: 0.4832
Epoch [8/15]: Loss: 0.6433
Epoch [9/15]: Loss: 0.7574
Epoch [10/15]: Loss: 0.4642
Validation Loss at Epoch 10: 80.7100
Epoch [11/15]: Loss: 0.5810
Epoch [12/15]: Loss: 0.4366
Epoch [13/15]: Loss: 0.4461
Epoch [14/15]: Loss: 0.4041
Test Accuracy (top-1): 34.95%
Finished Training. Total elapsed time: 86.9 min


In [12]:
torch.save(model.state_dict(), "/content/drive/MyDrive/models/flowersv1")
print("Saved PyTorch Model State to path")

Saved PyTorch Model State to path


Training 2:
* Test Accuracy (top-1): 34.95%
* Total elapsed time: 86.9 min

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = FlowerClassifier().to(device)
net.load_state_dict(torch.load("/content/drive/MyDrive/models/flowersv1"))

<All keys matched successfully>